In [8]:
import requests

url = "https://github.com/daivikpurani/LiveLikeALocal/blob/main/Data%20Scraping%2Foutput.txt"  # Replace with actual URL
response = requests.get(url)

if response.status_code == 200:
    text = response.text
    print(text)
else:
    print("Failed to fetch the file.")








<!DOCTYPE html>
<html
  lang="en"
  
  data-color-mode="auto" data-light-theme="light" data-dark-theme="dark"
  data-a11y-animated-images="system" data-a11y-link-underlines="true"
  
  >



  <head>
    <meta charset="utf-8">
  <link rel="dns-prefetch" href="https://github.githubassets.com">
  <link rel="dns-prefetch" href="https://avatars.githubusercontent.com">
  <link rel="dns-prefetch" href="https://github-cloud.s3.amazonaws.com">
  <link rel="dns-prefetch" href="https://user-images.githubusercontent.com/">
  <link rel="preconnect" href="https://github.githubassets.com" crossorigin>
  <link rel="preconnect" href="https://avatars.githubusercontent.com">

  


  <link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/light-74231a1f3bbb.css" /><link crossorigin="anonymous" media="all" rel="stylesheet" href="https://github.githubassets.com/assets/dark-8a995f0bacd4.css" /><link data-color-theme="dark_dimmed" crossorigin="anonymous" m

In [9]:
pip install -U langchain-openai


Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install langchain openai pinecone-client tiktoken


Note: you may need to restart the kernel to use updated packages.


In [18]:

pip install langchain-pinecone


Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install -U langchain-huggingface


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [46]:
import os
import requests
import tempfile

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec

# Set your Pinecone API key
os.environ["PINECONE_API_KEY"] = "pcsk_2qBhKx_FeNNTv2hf481CuEwNN985MN6LmY7xTrQzdULNiQNqvUWjVFBDicbm9Qncjc9AUc"

# Step 1: Download the GitHub file
url = "https://raw.githubusercontent.com/daivikpurani/LiveLikeALocal/main/Data%20Scraping/output.txt"
response = requests.get(url)
if response.status_code != 200:
    raise Exception("Failed to fetch the file.")

# Step 2: Save to a temp file
with tempfile.NamedTemporaryFile(delete=False, suffix=".txt") as temp_file:
    temp_file.write(response.content)
    temp_path = temp_file.name

# Step 3: Load text with LangChain
loader = TextLoader(temp_path, encoding="utf-8")
documents = loader.load()

# Step 4: Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(documents)

# Step 5: Use local HuggingFace embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Step 6: Connect to Pinecone
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
index_name = "travelagent"

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # 384 for MiniLM
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

# Step 7: Upload to Pinecone
vectorstore = PineconeVectorStore.from_documents(
    docs,
    embedding=embeddings,
    index_name=index_name,
    namespace="default"
)


In [83]:
results = vectorstore.similarity_search("What’s something unique to do in SF on a Wednesday?", k=3)
for i, doc in enumerate(results):
    print(f"\nResult {i+1}:\n")
    print(doc.page_content[:700])  




Result 1:

LI: $2 Taco Night + Happy Hour Comedy at Skylark (SF)
LI: Black Hammer’s Free “Hopped Up” Trivia Night (Every Wednesday)
LI: 80,000 Free Tulips Are Coming to SF’s Union Square (2025)
LI: SF’s Biggest Rooftop “Holi” Festival of Colors (2025)
LI: Grand Opening of SF’s New 10,000 sqft “Museum of Failure” (Fisherman’s Wharf)
LI: 2025 San Francisco Sail Grand Prix (March 22-23)
LI: SF’s Easter In the Park & Hunky Jesus Contest 2025 (Dolores Park)
LI: Castro Night Market (SF)
LI: 2025 Holi Festival at City Center (San Ramon)
LI: Spring Equinox Inner Sunset Art Walk (SF)
LI: 2025 Oakland Restaurant Week (March 20-30)
LI: SF’s 2025 Cherry Blossom Festival (April 12-20)
LI: SF’s St. Patrick’s Day Cel

Result 2:

LI: $2 Taco Night + Happy Hour Comedy at Skylark (SF)
LI: Black Hammer’s Free “Hopped Up” Trivia Night (Every Wednesday)
LI: 80,000 Free Tulips Are Coming to SF’s Union Square (2025)
LI: SF’s Biggest Rooftop “Holi” Festival of Colors (2025)
LI: Grand Opening of SF’s New 10,0